In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
from io import BytesIO
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder

from kaggle_camera_model_id_lib.utils import PechkaBot, ImageList, NpzFolder, NCrops, TifFolder, TifFolderExFiles
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead, IEEEfcn, ResNetFC, ResNetX, FatNet1
from kaggle_camera_model_id_lib.models import InceptionResNetV2fc, InceptionResNetV2fcSmall
from kaggle_camera_model_id_lib.utils import jpg_compress, equalize_v_hist, hsv_convert
from kaggle_camera_model_id_lib.utils import scale_crop_pad, gamma_correction
from kaggle_camera_model_id_lib.utils import patch_quality_dich, n_random_crops, n_pseudorandom_crops
from kaggle_camera_model_id_lib.models import DANet, ResNetFeatureExtractor, AvgFcClassifier, FCDiscriminator
from kaggle_camera_model_id_lib.models import AvgClassifier

import imgaug as ia
from imgaug import augmenters as iaa

In [2]:
val_path = '/home/mephistopheies/storage2/data/camera-model-id/val/'
test_path = '/home/mephistopheies/storage2/data/camera-model-id/raw/test/'
model_path = '/home/mephistopheies/storage2/data/camera-model-id/models/resnet34fc/gan/FCDiscriminator_AvgClassifier512/var2/checkpoint.tar'
out_dir = '/home/mephistopheies/storage2/data/camera-model-id/submit/'

model_type_fe = 'resnet34_fe'
model_type_d = 'FCDiscriminator'
model_type_c = 'AvgClassifier512'

n_classes = 10
crop_size = 256
step = 128
num_workers = 1

do_random_aug_kaggle = True
p_random_aug_kaggle = 0.5
do_hard_aug = False
p_hard_aug = 0.5

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

scale_05 = lambda img: scale_crop_pad(img, 0.5)
scale_08 = lambda img: scale_crop_pad(img, 0.8)
scale_15 = lambda img: scale_crop_pad(img, 1.5)
scale_20 = lambda img: scale_crop_pad(img, 2.0)
gamma_08 = lambda img: gamma_correction(img, 0.8)
gamma_12 = lambda img: gamma_correction(img, 1.2)
jpg_70 = lambda img: jpg_compress(img, (70, 71))
jpg_90 = lambda img: jpg_compress(img, (90, 91))
augs = [scale_05, scale_08, scale_15, scale_20, gamma_08, gamma_12, jpg_70, jpg_90]

blur = iaa.GaussianBlur(sigma=(0, 2))
sharpen = iaa.Sharpen(alpha=(0, 1), lightness=(0.5, 2))
emboss = iaa.Emboss(alpha=(0, 1), strength=(0, 2))
contrast_normalization = iaa.ContrastNormalization(alpha=(0.7, 1.3))
hard_aug = iaa.OneOf([blur, sharpen, emboss, contrast_normalization])
sometimes = iaa.Sometimes(p_hard_aug, hard_aug)


def random_aug_kaggle(img, p=0.5):
    if np.random.rand() < p:
        return random.choice(augs)(img)
    return img

def aug_train(img):
    if min(img.size) > crop_center_size:
        return random_flip(random_crop(center_crop(img)))
    return random_flip(random_crop(img))

def aug_optional(img):
    if do_hard_aug:
        img = Image.fromarray(sometimes.augment_image(np.array(img)))

    if do_random_aug_kaggle:
        img = random_aug_kaggle(img, p_random_aug_kaggle)
    return img

In [3]:
model_factory = {
    'resnet34_fe': lambda: ResNetFeatureExtractor(models.resnet.BasicBlock, [3, 4, 6, 3], load_resnet='resnet34'),
    'AvgFcClassifier': lambda n_classes: AvgFcClassifier(n_classes),
    'FCDiscriminator': lambda: FCDiscriminator(),
    'AvgClassifier512': lambda n_classes: AvgClassifier(n_classes, 512)
}

model = DANet(
        model_factory[model_type_fe](),
        model_factory[model_type_d](),
        model_factory[model_type_c](n_classes))


checkpoint = torch.load(model_path)
state = checkpoint['model']
model.load_state_dict(state)
class_to_idx = checkpoint['class_to_idx']
idx2class = dict([(v, k) for (k, v) in class_to_idx.items()])
epoch_log = checkpoint['trainin_log']
for k, v in sorted(epoch_log[-1].items(), key=lambda t: t[0]):
    print('  %s: %0.6f\n' % (k, v))
del(checkpoint)
model = model.cuda()
model = model.eval()

  acc_train_c: 0.995298

  acc_train_d: 0.995153

  acc_train_g: 0.998333

  acc_val: 0.974519

  loss_train_c: 0.014187

  loss_train_d: 0.014571

  loss_train_g: 0.007592

  loss_val: 0.086873

  time: 536.591299



In [4]:
batch_size = 15

ds_val = NpzFolder(
    val_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(img, crop_size=crop_size, step=step)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(aug_optional(Image.fromarray(crop)))) 
                                                     for crop in crops]))
    ]),
    target_transform=transforms.Compose([
            transforms.Lambda(lambda y: [y]*int(np.floor(1 + (512 - crop_size)/step))**2),
            transforms.Lambda(lambda ylist: torch.LongTensor(ylist))
    ]))
val_loader = torch.utils.data.DataLoader(    
    ds_val,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=1, 
    pin_memory=True)


acc_val_batch = 0
for ix_batch, (X, Y) in tqdm_notebook(enumerate(val_loader), total=int(len(ds_val.imgs)/batch_size)):
    bs, ncrops, c, h, w = X.shape
    X = X.view(-1, c, h, w)
    Y = Y.view(ncrops*bs)

    X_var = Variable(X.cuda(), volatile=True)
    Y_var = Variable(Y.cuda(), volatile=True)

    log_p = model(X_var, mode='c')

    acc_val_batch += ((log_p.max(1)[1] == Y_var).float().sum()/Y_var.shape[0]).data[0]

acc_val_batch /= ix_batch + 1
print(acc_val_batch)


0.9758024575312932


In [5]:
batch_size = 15

ds_val = NpzFolder(
    val_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(img, crop_size=crop_size, step=step)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(aug_optional(Image.fromarray(crop)))) 
                                                     for crop in crops]))
    ]))
val_loader = torch.utils.data.DataLoader(    
    ds_val,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=1, 
    pin_memory=True)


p_val = []
acc_val_batch = 0
for ix_batch, (X, Y) in tqdm_notebook(enumerate(val_loader), total=int(len(ds_val.imgs)/batch_size)):
    bs, ncrops, c, h, w = X.shape
    X = X.view(-1, c, h, w)
    X_var = Variable(X.cuda(), volatile=True)
    Y_var = Variable(Y.cuda(), volatile=True)
    log_p = model(X_var, mode='c')
    log_p = log_p.view(bs, ncrops, -1)
    p = F.softmax(log_p, dim=2)
    p = p.prod(dim=1).pow(1/p.shape[1])
    acc_val_batch += ((p.max(1)[1] == Y_var).float().sum()/Y_var.shape[0]).data[0]
    p_val.append(p.cpu().data.numpy())

p_val = np.vstack(p_val)
acc_val_batch /= ix_batch + 1
print(acc_val_batch)


0.991555559237798


In [6]:
batch_size = 1

ds_test = TifFolderExFiles(
    test_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(np.array(img), crop_size=crop_size, step=step)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
    ]))

test_loader = torch.utils.data.DataLoader(    
    ds_test,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=num_workers, 
    pin_memory=True)

res = []
p_test = {}
for X, Y, files in tqdm_notebook(test_loader, total=int(len(ds_test.imgs)/batch_size)):
    files = list(map(lambda s: os.path.basename(s), files))
    bs, ncrops, c, h, w = X.shape
    X = X.view(-1, c, h, w)
    X_var = Variable(X.cuda(), volatile=True)
    log_p = model(X_var, mode='c')
    log_p = log_p.view(bs, ncrops, -1)
    p = F.softmax(log_p, dim=2) #.mean(dim=1)
    p = p.prod(dim=1).pow(1/p.shape[1])
    ix_argmax = p.max(1)[1].cpu().data.numpy()
    res.extend(list(zip(files, [idx2class[ix] for ix in ix_argmax])))

    for ix in range(len(files)):
        p_test[files[ix]] = [(idx2class[i], x) for (i, x) in enumerate(p[ix, :].cpu().data.numpy())]        

In [7]:
pd.Series([v for (k, v) in res]).value_counts()

Motorola-X              347
Motorola-Droid-Maxx     298
iPhone-6                281
LG-Nexus-5x             274
Samsung-Galaxy-Note3    268
Samsung-Galaxy-S4       266
HTC-1-M7                265
Motorola-Nexus-6        263
Sony-NEX-7              224
iPhone-4s               154
dtype: int64

In [9]:
with open(os.path.join(out_dir, 'submit__unalt.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        if '_unalt' in fname:
            f.write('%s,%s\n' % (fname, c))
        else:
            f.write('%s,%s\n' % (fname, 'no_class'))
            
            
with open(os.path.join(out_dir, 'submit__manip.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        if '_manip' in fname:
            f.write('%s,%s\n' % (fname, c))
        else:
            f.write('%s,%s\n' % (fname, 'no_class'))
            
            
with open(os.path.join(out_dir, 'submit.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        f.write('%s,%s\n' % (fname, c))